# Baseline Experiment - Pre-optimized Santa 2025 Submission

This notebook establishes the baseline by:
1. Loading the pre-optimized santa-2025.csv
2. Validating no overlaps
3. Calculating the total score
4. Creating the submission file

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import shutil
import os

getcontext().prec = 30

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# ChristmasTree class definition
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        # 15-vertex polygon definition
        initial_polygon = Polygon([
            (0.0, 0.8),      # Tip
            (0.125, 0.5),    # Right top tier
            (0.0625, 0.5),
            (0.2, 0.25),     # Right mid tier
            (0.1, 0.25),
            (0.35, 0.0),     # Right base
            (0.075, 0.0),    # Right trunk
            (0.075, -0.2),
            (-0.075, -0.2),  # Left trunk
            (-0.075, 0.0),
            (-0.35, 0.0),    # Left base
            (-0.1, 0.25),    # Left mid tier
            (-0.2, 0.25),
            (-0.0625, 0.5),  # Left top tier
            (-0.125, 0.5),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Load the pre-optimized submission
submission_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(submission_path)
print(f"Loaded submission with {len(df)} rows")
print(f"Expected rows: {sum(range(1, 201))} (1+2+...+200)")
print(f"\nFirst 10 rows:")
print(df.head(10))

Loaded submission with 20100 rows
Expected rows: 20100 (1+2+...+200)

First 10 rows:
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.37062226926408

In [ ]:
# Parse the submission - values are prefixed with 's'
def parse_value(val):
    """Remove 's' prefix and return as string"""
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    """Load all trees for configuration N"""
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Test loading N=1
trees_1 = load_trees_for_n(df, 1)
print(f"N=1: {len(trees_1)} tree(s)")
print(f"  Position: ({trees_1[0].center_x}, {trees_1[0].center_y})")
print(f"  Angle: {trees_1[0].angle}")

In [ ]:
# Check if N=1 is at 45 degrees (optimal for minimizing bounding box)
print(f"N=1 angle: {trees_1[0].angle}")
if abs(float(trees_1[0].angle) - 45.0) < 0.1:
    print("✓ N=1 is at optimal 45-degree rotation")
else:
    print(f"⚠ N=1 is NOT at 45 degrees - potential improvement opportunity!")

In [ ]:
# Overlap detection function
def has_overlap(trees):
    """Check if any trees overlap (excluding touching)"""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                # Check if it's a real overlap (not just touching)
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:  # Small tolerance
                    return True
    return False

print("Overlap detection function defined")

In [ ]:
# Validate no overlaps for all N (this may take a while)
print("Validating no overlaps for all configurations...")
overlap_count = 0
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    if has_overlap(trees):
        print(f"  ⚠ Overlap detected in N={n}!")
        overlap_count += 1
    if n % 50 == 0:
        print(f"  Checked N=1 to {n}...")

if overlap_count == 0:
    print("✓ No overlaps detected in any configuration!")
else:
    print(f"⚠ Found {overlap_count} configurations with overlaps!")

In [ ]:
# Calculate the total score
def get_bounding_box_side(trees):
    """Get the side length of the bounding square for a set of trees"""
    all_points = []
    for tree in trees:
        coords = np.asarray(tree.polygon.exterior.xy).T
        all_points.append(coords)
    all_points = np.concatenate(all_points)
    
    min_coords = all_points.min(axis=0)
    max_coords = all_points.max(axis=0)
    
    # Bounding box side is the max of width and height
    side = max(max_coords - min_coords)
    return side

def calculate_total_score(df):
    """Calculate total score: sum(s_n^2 / n) for n=1 to 200"""
    total = 0
    scores_per_n = []
    
    for n in range(1, 201):
        trees = load_trees_for_n(df, n)
        side = get_bounding_box_side(trees)
        score_n = (side ** 2) / n
        total += score_n
        scores_per_n.append((n, side, score_n))
        
        if n <= 5 or n % 50 == 0:
            print(f"  N={n}: side={side:.6f}, score_contribution={score_n:.6f}")
    
    return total, scores_per_n

print("Calculating total score...")
total_score, scores_per_n = calculate_total_score(df)
print(f"\n=== TOTAL SCORE: {total_score:.6f} ===")

In [ ]:
# Analyze score contributions
scores_df = pd.DataFrame(scores_per_n, columns=['n', 'side', 'score_contribution'])
scores_df['efficiency'] = scores_df['n'] / (scores_df['side'] ** 2)  # trees per unit area

print("Top 10 highest score contributions (worst efficiency):")
print(scores_df.nlargest(10, 'score_contribution')[['n', 'side', 'score_contribution', 'efficiency']])

print("\nTop 10 lowest efficiency (trees per unit area):")
print(scores_df.nsmallest(10, 'efficiency')[['n', 'side', 'score_contribution', 'efficiency']])

In [ ]:
# Copy to submission directory
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(submission_path, '/home/submission/submission.csv')
print(f"Copied submission to /home/submission/submission.csv")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows")

In [ ]:
# Summary
print("="*60)
print("BASELINE SUMMARY")
print("="*60)
print(f"Total Score: {total_score:.6f}")
print(f"Target Score: 68.919154")
print(f"Gap to Target: {total_score - 68.919154:.6f}")
print(f"Overlaps: {overlap_count}")
print(f"N=1 Angle: {float(trees_1[0].angle):.1f} degrees")
print("="*60)